In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models import efficientnet_v2_s  # 修改1：导入EfficientNetV2-s
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, matthews_corrcoef
import seaborn as sns
from tqdm import tqdm
import random

# 设置随机种子保证可重复性
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 自定义数据集类（保持不变）
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append((os.path.join(cls_dir, img_name), self.class_to_idx[cls_name]))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# 数据增强变换（保持不变）
train_transform = transforms.Compose([
    transforms.Resize((384, 384)),  # 修改2：调整输入尺寸为EfficientNetV2推荐尺寸
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((384, 384)),  # 修改3：验证集同样调整尺寸
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 创建数据集和数据加载器（路径需自行确认）
train_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset4\train', transform=train_transform)
val_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset4\val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

# 修改模型定义部分（关键修改）
class EfficientNetV2(nn.Module):
    def __init__(self, num_classes=4):
        super(EfficientNetV2, self).__init__()
        # 加载预训练的EfficientNetV2-M并修改最后的全连接层
        self.model = efficientnet_v2_s(pretrained=True)
        # 修改4：冻结特征提取层（可选）
        for param in self.model.features.parameters():
            param.requires_grad = False
        # 修改5：替换分类器
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# 初始化模型、损失函数和优化器（修改6：使用新模型类）
model = EfficientNetV2(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 修改7：调整学习率

# 训练和验证函数（保持不变）
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(dataloader)
    train_acc = 100 * correct / total
    
    return train_loss, train_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = running_loss / len(dataloader)
    val_acc = 100 * correct / total
    cm = confusion_matrix(all_labels, all_preds)
    
    return val_loss, val_acc, cm

# 训练参数（保持不变）
num_epochs = 500
best_val_acc = 0.0
best_model_state = None
train_losses = []
train_accs = []
val_losses = []
val_accs = []

# 训练循环（保持不变）
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, cm = validate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()
        torch.save(best_model_state, 'best_model_4class.pth')
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# 绘制训练曲线（保持不变）
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Curve')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves_4class.png')
plt.show()

# 加载最佳模型（保持不变）
model.load_state_dict(torch.load('best_model_4class.pth'))
model.eval()

# 验证集评估（保持不变）
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs.data, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# 计算混淆矩阵（保持不变）
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=train_dataset.classes, 
            yticklabels=train_dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_4class.png')
plt.show()

# 计算分类报告（保持不变）
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

# 计算MCC（修改为多分类版本）
mcc = matthews_corrcoef(all_labels, all_preds)

# 输出结果（保持原格式）
print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

# 每个类别的详细指标（保持原格式）
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes, output_dict=True)
for i, class_name in enumerate(train_dataset.classes):
    cls_report = report[class_name]
    tn = cm[i, i]
    fp = cm[:, i].sum() - tn
    fn = cm[i, :].sum() - tn
    tp = tn  # 对角线元素即为TP
    
    specificity = (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn) / (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn + fp) if (cm.sum() - cm[i, :].sum() - cm[:, i].sum() + tn + fp) != 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    
    print(f"\nClass: {class_name}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-score: {cls_report['f1-score']:.4f}")

Using device: cuda


D:\Anaconda\anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00,  9.61it/s]


Train Loss: 1.3875, Train Acc: 36.20%
Val Loss: 1.3197, Val Acc: 41.38%

Epoch 2/500


...

Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00,  9.76it/s]


Train Loss: 0.7724, Train Acc: 73.89%
Val Loss: 1.3551, Val Acc: 52.41%

Epoch 500/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00,  9.84it/s]


Train Loss: 0.7124, Train Acc: 71.51%
Val Loss: 1.2165, Val Acc: 52.41%



Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.50      0.58        26
           2       0.50      0.47      0.48        34
         3-4       0.63      0.67      0.65        61
           5       0.59      0.71      0.64        24

    accuracy                           0.60       145
   macro avg       0.60      0.59      0.59       145
weighted avg       0.60      0.60      0.60       145


Best Validation Accuracy: 60.00%
Matthews Correlation Coefficient (MCC): 0.4325

Class: 0
Sensitivity (Recall): 0.5000
Specificity: 0.9496
F1-score: 0.5778

Class: 2
Sensitivity (Recall): 0.4706
Specificity: 0.8559
F1-score: 0.4848

Class: 3-4
Sensitivity (Recall): 0.6721
Specificity: 0.7143
F1-score: 0.6508

Class: 5
Sensitivity (Recall): 0.7083
Specificity: 0.9008
F1-score: 0.6415
